# Story Generator - Winnie the Pooh 

## Part 2: Simple DSPy Retriever and Module

[1. Imports and environment](#1-imports-and-environment)

[2. Chroma retriever](#2-chroma-retriever)

[3. DSPy module](#3-dspy-module)

[4. Testing StoryGenerator](#4-testing-storygenerator)


### 1. Imports and environment

In [1]:
#pip install dspy-ai openai chromadb sentence_transformers spacy textstat

In [2]:
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM
import chromadb
from chromadb.utils import embedding_functions
import dotenv
import os
import spacy


# Establish paths
CHROMA_PATH = '../data/chroma_db'
DB_COLLECTION = "winnie_the_pooh"
default_ef = embedding_functions.DefaultEmbeddingFunction()

# Set up OpenAI API key
dotenv.load_dotenv()
#openai_key = os.getenv('OPENAI_API_KEY')

/Users/Lila/Desktop/SpringBoard/Github/bedtime-stories-ai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# List all collections in the Chroma database
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collections = chroma_client.list_collections()
print(collections)

[Collection(name=winnie_the_pooh)]


### 2. Chroma retriever

In [4]:
# Configure OpenAI as the language model
llm = dspy.OpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=1.0)

# Set up Chroma client and retriever
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = chroma_client.get_collection(DB_COLLECTION)

# Set up ChromadbRM as the retriever model
chroma_retriever = ChromadbRM(
    collection_name=DB_COLLECTION, 
    persist_directory=CHROMA_PATH, 
    embedding_function=default_ef,
    )

# Configure DSPy settings
dspy.settings.configure(lm=llm, rm=chroma_retriever)

In [5]:
#example of calling retriever
results = chroma_retriever("honey")
len(results)
results[0]

{'id': '3ac39b49-2cbd-4a32-b871-51b1fd945193',
 'score': 1.084073543548584,
 'long_text': 'then he got up, and said: "And the only reason for making honey is so as I can eat it." So he began to climb the tree. He climbed and he climbed and he climbed, and as he climbed he sang a little',
 'metadatas': {'author': 'A. A. Milne',
  'chapter': 1.0,
  'chunk': 17.0,
  'title': 'Winnie the Pooh'}}

### 3. DSPy module

In [6]:

class GenerateStory(dspy.Signature):
    """Generate a Winnie the Pooh style story."""
    name = dspy.InputField()
    prompt = dspy.InputField(desc="details to include in the story.")
    context = dspy.InputField(desc="relevant passages from Winnie the Pooh stories and story structure.")
    story = dspy.OutputField(desc="generate a one-minute story for a child. Name is the main character who is friends with Pooh, and finish the story with 'The End.'")


class StoryGenerator(dspy.Module):
    def __init__(self, chroma_retriever):
        super().__init__()
        self.retriever = chroma_retriever
        self.generate = dspy.ChainOfThought(GenerateStory)

    def forward(self, name, prompt):
        retrieved = self.retriever(prompt, k=8)
        retrieved_context = [doc.long_text for doc in retrieved]
        context = "\n".join(retrieved_context)
        
        result = self.generate(context=context, prompt=prompt, name=name)
        return dspy.Prediction(story=result.story)



### 4. Testing StoryGenerator 

In [7]:
# Create an instance of the StoryGenerator
story_gen = StoryGenerator(chroma_retriever)


In [8]:
name= 'Rose'
prompt = "They go into town and meet many new friends and have a picnic."

new_story2 = story_gen(name, prompt)
print(new_story2.story)

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Once upon a time, in the gentle warmth of a sunny morning, Rose, a little bear with a bright yellow coat, decided it was the perfect day for a picnic. So, she hurried over to Pooh's house. “Pooh, would you like to come to town for a picnic?” she asked eagerly.

“Oh, I do love picnics!” said Pooh, his eyes twinkling. “But we must gather everyone first!”

Off they went, hand in paw, singing a cheerful tune until they reached Piglet's house. “Piglet! We’re going to town for a picnic!” Rose shouted.

“A picnic? Oh, how delightful!” squeaked Piglet, jumping with joy. “I’ll come too!”

Next, they visited Eeyore, who was busy under his favorite tree. “Eeyore! We’re having a picnic in town, would you join us?” asked Rose.

“Why not?” Eeyore replied with a shrug. “It’s not like I have anything else to do. Might even find a new friend or two.”

With their delightful troop gathered, they began their adventure to town. Upon arriving, they met a friendly rabbit named Benny, who was hopping joyfully